In [ ]:
boundary_id = 70704

In [ ]:
import boto3
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, box, Polygon
from rasterio.transform import rowcol
import rasterio
from samgeo import tms_to_geotiff
import numpy as np
import leafmap
import rioxarray
from deepforest import main, get_data
from deepforest.visualize import plot_results
from botocore.exceptions import NoCredentialsError, ClientError
import re
import os

access_key_id = os.environ.get('access_key_id')
secret_access_key = os.environ.get('secret_access_key')
region = os.environ.get('region')
bucket_name = os.environ.get('bucket_name')

global s3_client
if access_key_id and region and secret_access_key:
    s3_client = boto3.client('s3',
        aws_access_key_id=access_key_id,
        aws_secret_access_key=secret_access_key,
        region_name=region
    )


model = main.deepforest()
model.load_model(model_name="weecology/deepforest-tree")

def download_kml(bucket_name, boundary_id, local_path="boundary.kml"):
    key = f"ALP_kmls/{boundary_id}.kml"
    s3_client.download_file(bucket_name, key, local_path)
    print(f"Downloaded {key} to {local_path}")
    return local_path

def ensure_min_area(kml_path, min_area=9500, epsg_projected=32644):
    # Read and reproject to projected CRS for area calculation
    gdf = gpd.read_file(kml_path, driver="KML")
    gdf = gdf.to_crs(epsg=epsg_projected)

    # Compute current area
    current_area = gdf.geometry.area.sum()

    if current_area < min_area:
        # Estimate required buffer radius (approximate)
        deficit = min_area - current_area
        buffer_dist = (deficit / current_area) ** 0.5 * 10  # heuristic scaling

        # Apply buffer iteratively until area ≥ min_area
        while gdf.geometry.area.sum() < min_area:
            gdf["geometry"] = gdf.buffer(buffer_dist)
            buffer_dist *= 1.1  # increase buffer if still not enough

    # Convert back to WGS84
    gdf = gdf.to_crs(epsg=4326)
    return gdf

def get_tiff(file):
    raster = rioxarray.open_rasterio(file)
    return raster

def get_tree_crowns(gdf, image, model):
    image.shape
    image_transpo = image.transpose('x','y','band')

    if sum([len(x) for x in np.where(image != 0)]) > 0:

        pred_boxes = model.predict_image(image=image_transpo.values)

        if isinstance(pred_boxes, pd.DataFrame) and pred_boxes is not None:

            pred_boxes['new_geometry'] = None
            pred_boxes['new_geometry'] = pred_boxes.geometry.apply(lambda coord:Polygon([(image_transpo[int(y)-1,int(x)-1,:].coords['x'].values.item(),image_transpo[int(y)-1,int(x)-1,:].coords['y'].values.item()) for x,y in list(coord.exterior.coords)]))
            pred_boxes_new = pred_boxes.copy()
            pred_boxes_new = pred_boxes_new.drop('geometry', axis=1).rename(columns= {'new_geometry':'geometry'})


            # Google Images Projection - 3857
            pred_boxes_new.set_crs(crs = 3857, inplace=True)
            pred_boxes_new = pred_boxes_new.to_crs(crs = 4326)
            confidence_factor = pred_boxes_new.copy()

            conf_preds_for_crs = pred_boxes_new.to_crs("EPSG:3857")
            conf_preds_for_crs['area_m2'] = conf_preds_for_crs.geometry.area

            sel_forest_calss = conf_preds_for_crs.copy()
            return sel_forest_calss
        else:
            return None
    else:
        return None

def geoms_to_pixel_bbox(gdf, raster_path):
    """
    Overwrite gdf xmin,ymin,xmax,ymax by converting geometry bounds into pixel coords
    using the raster transform. Returns a new GeoDataFrame with pixel bbox columns.
    """

    with rasterio.open(raster_path) as src:
        transform = src.transform
        width = src.width
        height = src.height
        xmins, ymins, xmaxs, ymaxs = [], [], [], []
        for geom in gdf.geometry:
            minx, miny, maxx, maxy = geom.bounds
            # convert map coords -> row/col
            row_min, col_min = rowcol(transform, minx, maxy)  # careful ordering
            row_max, col_max = rowcol(transform, maxx, miny)
            # clip to image bounds
            xmin = np.clip(col_min, 0, width - 1)
            ymin = np.clip(row_min, 0, height - 1)
            xmax = np.clip(col_max, 0, width - 1)
            ymax = np.clip(row_max, 0, height - 1)
            xmins.append(xmin)
            ymins.append(ymin)
            xmaxs.append(xmax)
            ymaxs.append(ymax)
        out = gdf.copy()
        out["xmin"] = xmins
        out["ymin"] = ymins
        out["xmax"] = xmaxs
        out["ymax"] = ymaxs
        return out

def user_rois_to_bbox(user_rois, target_crs="EPSG:3857", label="Tree", raster_path=None):

    if user_rois["type"] == "FeatureCollection":
        features = user_rois["features"]
    else:
        features = [user_rois]

    user_gdf = gpd.GeoDataFrame(
        [feat.get("properties", {}) for feat in features],
        geometry=[shape(feat["geometry"]) for feat in features],
        crs="EPSG:4326"
    )

    user_gdf = user_gdf.to_crs(target_crs)

    user_gdf["xmin"] = user_gdf.bounds.minx
    user_gdf["ymin"] = user_gdf.bounds.miny
    user_gdf["xmax"] = user_gdf.bounds.maxx
    user_gdf["ymax"] = user_gdf.bounds.maxy

    user_gdf["label"] = label
    user_gdf["score"] = None
    user_gdf["area_m2"] = user_gdf.geometry.area

    cols = ["xmin", "ymin", "xmax", "ymax", "label", "score", "geometry", "area_m2"]
    return geoms_to_pixel_bbox(user_gdf, raster_path)



def upload_to_s3(local_path, s3_key):
    global s3_client, bucket_name
    try:
        # Check if file exists locally
        if not os.path.exists(local_path):
            print(f"Local file {local_path} does not exist")
            return None

        # Upload the file
        with open(local_path, "rb") as f:
            s3_client.put_object(Bucket=bucket_name.replace(" ", ""), Key=s3_key, Body=f, ACL='public-read')
            print(f"Successfully uploaded {local_path} to {s3_key}")

        # Return the S3 URL (https)
        s3_url = f"https://{bucket_name}.s3.amazonaws.com/{s3_key}"
        print(f"Successfully uploaded {local_path} to {s3_url}")
        return s3_url

    except NoCredentialsError:
        print("AWS credentials not found. Please configure your AWS credentials.")
        return None
    except ClientError as e:
        print(f"Error uploading file: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None



##CODE STARTS

local_path = f"{boundary_id}.kml"
kml_path = download_kml(bucket_name, boundary_id, local_path)
gdf = ensure_min_area(kml_path, min_area=9500)
source = "Satellite"

kml_filename = os.path.splitext(os.path.basename(kml_path))[0]
kml_filename = re.sub(r'[^A-Za-z0-9_-]', '_', kml_filename)
osm_download = f"{kml_filename}.tif"
s3_key = f"satellite-images/{osm_download}"
minx, miny, maxx, maxy = gdf.total_bounds
centroid = gdf.geometry.centroid.iloc[0]

tms_to_geotiff(osm_download, bbox=[minx, miny, maxx, maxy], zoom=19, source=source, overwrite=True)

uploaded_url = upload_to_s3(osm_download, s3_key)

image = get_tiff(osm_download)

concaT_df = get_tree_crowns(gdf, image, model)
geome = concaT_df[(concaT_df.area_m2<=60) & (concaT_df.score>=0.15)].reset_index(drop=True)
geome['predicted_by'] = 'Model'

m = leafmap.Map()
m.add_basemap('SATELLITE', layers_control=True)
style = {
    "color": "#ff0000",
    "weight": 2,
    "fillOpacity": 0,
}
m.add_cog_layer(url=uploaded_url, name="Satellite",zoom_to_layer=True)
if not geome.empty:
    m.add_gdf(geome, layer_name="Predicted Trees")
m





Reading config file: /usr/local/lib/python3.11/site-packages/deepforest/data/deepforest_config.yml
Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth


100%|██████████| 130M/130M [00:27<00:00, 4.92MB/s] 
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Reading config file: /usr/local/lib/python3.11/site-packages/deepforest/data/deepforest_config.yml


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

In [ ]:
##RUN IT AFTER COMPLETING WITH THE MARKINGS

In [ ]:
drawn_features = m.user_rois
gdf_drawn = user_rois_to_bbox(drawn_features, target_crs="EPSG:3857", label="Tree", raster_path=osm_download)
gdf_drawn['labelled_by'] = 'user'
geome['labelled_by'] = 'model'
final_gdf = pd.concat([geome, gdf_drawn], ignore_index=True)
final_gdf["image_path"] = s3_key
csv_path = osm_download.split(".")[0]+"-annotations.csv"
csv_s3_key = f"annotations/{csv_path}"
final_gdf.to_csv(csv_path, index=False)
upload_to_s3(csv_path, csv_s3_key)

os.remove(local_path)
os.remove(csv_path)
os.remove(osm_download)
